## Lab 2: final challenges

__Вам предлагается решить задачу классификации сигналов или задачу классификации изображений. Или обе ;)__

__Выполнение этих заданий не является обязательным, но позитивно повлияет на вашу итоговую оценку. Успехов!__









### Part 5. Dogs classification (2+ points)
__Disclaimer__: Это опциональная часть задания. Здесь придется экспериментировать, подбирать оптимальную структуру сети для решения задачи и активно искать подскзаки в сети.

Предлагаем вам решить задачу классификации пород собак. Вы можете обучить сеть с нуля или же воспользоваться методом fine-tuning'а. Полезная ссылка на [предобученные модели](https://pytorch.org/docs/stable/torchvision/models.html).

Данные можно скачать [отсюда](https://www.dropbox.com/s/vgqpz2f1lolxmlv/data.zip?dl=0). Датасет представлен 50 классами пород собак, которые можно найти в папке train в соответствующих директориях. При сдаче данной части задания вместе с ноутбуком необходимо отправить .csv-файл с предсказаниями классов тестовой выборки в формате: <имя изображения>,<метка класса> по одному объекту на строку. Ниже приведите код ваших экспериментов и короткий вывод по их результатам.

Будут оцениваться качество классификации (accuracy) на тестовой выборке (2 балла) и проведенные эксперименты (1 балл).
Разбалловка следующая:
* $>=$93% - 2 points
* $>=$84% - 1.5 points
* $>=$70% - 0.75 points

# Отчет

Я взял предобученную на Imagenet ResNet-152, обучил ее методом fine-tuning, добавив линейный слой в конце. Также попробовал добавлять скедулеры и количество эпох, чтобы улучшать перфоманс модели. В результате получилась accuracy = 0.8838. После этого  я добавил аугментации (флипы, повороты) и благодаря этому точность выросла до 0.9181 на валидационных данных.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
from torchvision import transforms, models, datasets
import torch

image_std = [0.229, 0.224, 0.225]
image_mean = [0.485, 0.456, 0.406]
data_transforms = transforms.Compose(
    [transforms.RandomRotation(degrees = 20), transforms.Resize((224, 224)), transforms.ToTensor(),
     transforms.Normalize(image_mean, image_std), transforms.RandomHorizontalFlip()])
data_dir = '/content/drive/My Drive/data'
datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                          data_transforms)
split1 = int(len(datasets)*0.8)
split2 = len(datasets) - split1
train_dataset, val_dataset = torch.utils.data.random_split(datasets, [split1, split2])
image_datasets = {'train':train_dataset, 'test':val_dataset}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'test']}

In [8]:
from PIL import Image
img_path = '/content/drive/My Drive/data/test/'
test_dataset = []
test_nums = []
for i in range(1503):
  img = Image.open(str(img_path)+str(i)+".jpeg")
  img = data_transforms(img)
  test_dataset.append(img)
  test_nums.append(str(i)+".jpeg")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, 
                                          shuffle=False)


In [9]:

import torch.nn as nn
import torch
device = "cuda"
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features

for param in model_ft.parameters():
    param.requires_grad = False
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 50)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [10]:


def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    

    
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

    return model

In [11]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

Epoch 0/14
----------
train Loss: 1.9290 Acc: 0.5393
test Loss: 0.4729 Acc: 0.8729
Epoch 1/14
----------
train Loss: 0.9023 Acc: 0.7761
test Loss: 0.3113 Acc: 0.9062
Epoch 2/14
----------
train Loss: 0.7393 Acc: 0.7985
test Loss: 0.3359 Acc: 0.9097
Epoch 3/14
----------
train Loss: 0.6830 Acc: 0.8105
test Loss: 0.3468 Acc: 0.8958
Epoch 4/14
----------
train Loss: 0.6670 Acc: 0.8098
test Loss: 0.3248 Acc: 0.9056
Epoch 5/14
----------
train Loss: 0.6312 Acc: 0.8195
test Loss: 0.3474 Acc: 0.9042
Epoch 6/14
----------
train Loss: 0.6103 Acc: 0.8256
test Loss: 0.3073 Acc: 0.9104
Epoch 7/14
----------
train Loss: 0.5031 Acc: 0.8582
test Loss: 0.3192 Acc: 0.9125
Epoch 8/14
----------
train Loss: 0.5089 Acc: 0.8506
test Loss: 0.2994 Acc: 0.9139
Epoch 9/14
----------
train Loss: 0.4772 Acc: 0.8641
test Loss: 0.2966 Acc: 0.9153
Epoch 10/14
----------
train Loss: 0.5074 Acc: 0.8501
test Loss: 0.2780 Acc: 0.9181
Epoch 11/14
----------
train Loss: 0.5006 Acc: 0.8528
test Loss: 0.3205 Acc: 0.9056
Ep

In [12]:
preds_list = []
model_ft.eval()
for X_batch in test_loader:
  X_batch = X_batch.to(device)
  outputs = model_ft(X_batch)
  _, preds = torch.max(outputs, 1)
  preds_list.append(preds)

In [13]:
preds_all = torch.cat(preds_list)

In [16]:
idx_to_class = dict(list(
    zip(datasets.class_to_idx.values(), 
        map(int, datasets.class_to_idx.keys()))))


In [17]:
with open("answer_corr.csv", "w") as f:
  for i, sample in enumerate(preds_all):
    f.write("{}.jpeg".format(str(i)) +": " +
            str(idx_to_class[int(sample)]) + "\n")